Here I have imported the necessary libraries

In [12]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report,r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression

Reading the data from the given data.

In [13]:
train_data =[]
with open("train_data.txt",'r') as f:
  p = f.read()
  for row in p.strip().split('\n'):
    parts = row.split(' ::: ')
    train_data.append({'id': parts[0], 'title': parts[1], 'genre': parts[2], 'description': parts[3]})

train_df = pd.DataFrame(train_data)
test_data=[]
with open("test_data.txt","r") as f:
  h=f.read()
  for row in h.strip().split('\n'):
    parts = row.split(' ::: ')
    test_data.append({
        'id': parts[0],
        'title': parts[1],
        'description': parts[2]
    })
test_df = pd.DataFrame(test_data)
df_combined = pd.concat([train_df, test_df], axis = 0)
df_combined.head()

,id,title,genre,description
0,1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his doc...
1,2,Cupid (1997),thriller,A brother and sister with a past incestuous re...
2,3,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fiel...
3,4,The Secret Sin (1915),drama,To help their unemployed father make ends meet...
4,5,The Unrecovered (2007),drama,The film's title refers not only to the un-rec...


train the model using the pipeline

Prediction!!!

In [14]:
df_combined = df_combined.dropna(subset=["description", "genre"])

encoder = LabelEncoder()
df_combined["genre_encoded"] = encoder.fit_transform(df_combined["genre"])


vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_train = vectorizer.fit_transform(df_combined["description"])
y_train = df_combined["genre_encoded"]

model = LogisticRegression(max_iter=1000, class_weight='balanced')
model.fit(X_train, y_train)
X_test = vectorizer.transform(test_df["description"])


y_pred_encoded = model.predict(X_test)
y_pred_labels = encoder.inverse_transform(y_pred_encoded)


test_df["predicted_genre"] = y_pred_labels

#Output
print(test_df[["id", "title", "predicted_genre"]])


<ipython-input-14-3789268570>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_combined["genre_encoded"] = encoder.fit_transform(df_combined["genre"])


          id                           title predicted_genre
0          1            Edgar's Lunch (1998)           short
1          2        La guerra de papá (1977)           drama
2          3     Off the Beaten Track (2010)     documentary
3          4          Meu Amigo Hindu (2015)           drama
4          5               Er nu zhai (1955)           drama
...      ...                             ...             ...
54195  54196  "Tales of Light & Dark" (2013)          horror
54196  54197     Der letzte Mohikaner (1965)             war
54197  54198             Oliver Twink (2007)         romance
54198  54199               Slipstream (1973)           music
54199  54200       Curitiba Zero Grau (2010)           short

[54200 rows x 3 columns]
